## Item-based Collaborative Filtering Algorithm Based on Group Weighted Rating.

In [1]:
# I have used these 4 libraries.
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import math, joblib
import warnings
warnings.filterwarnings('ignore')

In [2]:
def uar(userId, train_pt_df):
    l = [x for x in list(train_pt_df.loc[userId]) if x != 0]
    if len(l)==0:
        return 2.5
    else:
        return sum(l)/len(l)


def iar(itemId, train_pt_df):
    l = [x for x in list(train_pt_df[itemId]) if x != 0]
    if len(l)==0:
        return 2.5
    else:
        return sum(l)/len(l)


def func(train_pt_df):
    train_pt_df = train_pt_df.T
    
    user_average_rating = {userId:uar(userId, train_pt_df) for userId in train_pt_df.index}
    item_average_rating = {itemId:iar(itemId, train_pt_df) for itemId in train_pt_df.columns}
    
    git = {}
    for item in (train_pt_df.columns):
        l = train_pt_df[item].tolist()
        M = 3
        rp = [i for i in l if i >= M and i != 0]
        rq = [i for i in l if i < M and i != 0]
        left = 0
        right = 0
        for i in rp:
            left += i-M
        for i in rq:
            right += i-M
        git[item] = left - right
    
    gitd = {}
    for item in (train_pt_df.columns):
        if git[item] > 0:
            l = train_pt_df[item].tolist()
            M = 3
            rp = [i for i in l if i >= M and i != 0]
            if len(rp) == 0:
                gitd[item] = 0
            else:
                gitd[item] = sum(rp)/len(rp)
        elif git[item] == 0:
            gitd[item] = 3
        else:
            l = train_pt_df[item].tolist()
            M = 3
            rq = [i for i in l if i < M and i != 0]
            if len(rq) == 0:
                gitd[item] = 0
            else:
                gitd[item] = sum(rq)/len(rq)
        
    new_train_pt_df = train_pt_df

    for item in tqdm(train_pt_df.columns):
        for user in train_pt_df.index:
            if train_pt_df[item][user] == 0:
    #             r = [x for x in list(train_pt_df[item]) if x != 0]
    #             ri_bar = item_average_rating[item]
    #             r = [i-ri_bar for i in r]
    #             ru = user_average_rating[user] + (sum(r)/len(r))

    #             u = [x for x in list(train_pt_df.iloc[user-1]) if x != 0]
    #             ui_bar = user_average_rating[user]
    #             u = [i-ui_bar for i in u]
    #             ri = item_average_rating[item] + (sum(u)/len(u))
                one = user_average_rating[user]
                two = item_average_rating[item]
                three = gitd[item]
                new_train_pt_df[item][user] = three #math.sqrt(three)
    return new_train_pt_df.T

In [3]:
def predict_rating_IB(userId, itemId, train_pt_df, sm_df, K):
    try:
        users_ratings = train_pt_df[userId]  # ratings of all items for user 'userId'
        items_similarities = sm_df[itemId]  # cosine similarities of item 'itemId' with all other items.
    except:
        return -1
    
    # Consider only highest K item similarities. (Here, similarity of the same item will also not considered).
    d = dict(items_similarities) # {itemId : item_similarity}
    d_sorted = {k: v for k, v in sorted(d.items(), key=lambda item: item[1], reverse=True)}
    drop_rest_users = list(d_sorted.keys())
    drop_rest_users = drop_rest_users[K:]
    users_ratings = users_ratings.drop(drop_rest_users)
    items_similarities = items_similarities.drop(drop_rest_users)
    
    # Not consider items whose ratings are NA.
    drop_empty_ratings = users_ratings[users_ratings == 0].index
    users_ratings = users_ratings.drop(drop_empty_ratings)
    items_similarities = items_similarities.drop(drop_empty_ratings)
    
    global coverage
    # # If I encountered with coverage problem i.e, no similar items exist after threshold. Then take aveage rating of the movie.
    if len(users_ratings) == 0 or len(items_similarities) == 0:
        coverage = coverage + 1
        return -1
    else:
        # Normalize all the similarities of all the items.
        items_similarities_sum = sum(items_similarities)
        items_similarities = [i/items_similarities_sum for i in items_similarities]

        # Linearly interpolate the Active User 'userId' rated items by corresponding normalized similarities.
        linear_interpolation = np.dot(users_ratings, items_similarities)

        # return this predicted rating.
        if math.isnan(linear_interpolation) ==  False:
            return linear_interpolation
        else:
            return -1

In [4]:
# This function takes the training & testing dataframes and return the MAE.
def IB_MAE(train_df, test_df, K):
    # pivot the training dataframe and Transpose it.
    train_pt_df = pd.pivot_table(train_df, values='Rating', index='userId', columns='itemId').T
    # Replace the NA values with 0   (Note: I observed -> No user in the whole dataset have rated 0 to any movie)
    train_pt_df = train_pt_df.fillna(0)
    
    train_pt_df = func(train_pt_df)
    
    # Calculate the cosine similarities of item-item.
    sm_df = pd.DataFrame(cosine_similarity(train_pt_df), index=train_pt_df.index, columns=train_pt_df.index)
    # Not consider the similarity of same item while predicting the rating. eg: similarity of item 1 with item 1.
    np.fill_diagonal(sm_df.values, 0)
    
    # Actual Ratings of testing dataframe.
    actual_ratings  = list(test_df['Rating'])
    # Start predicting the ratings of testing dataframe.
    predicted_ratings = []
    for userId, itemId in tqdm(zip(test_df['userId'], test_df['itemId'])):
        predicted_ratings.append(predict_rating_IB(userId, itemId, train_pt_df, sm_df, K))
        
    # Ignore the case when predicted rating is -1. (Because no such items are available to predict the rating. #Coverage_Problem)
    new_actual_ratings = []
    new_predicted_ratings = []
    for i in range(0, len(predicted_ratings)):
        if predicted_ratings[i] <= 0:
            continue
        else:
            new_actual_ratings.append(actual_ratings[i])
            new_predicted_ratings.append(predicted_ratings[i])
    
    # return the MAE between Actual Ratings & Predicted Ratings.
    return mean_absolute_error(new_actual_ratings, new_predicted_ratings)

In [5]:
df = pd.read_csv('./Dataset/reviews.csv')
df = df.rename(columns={'RecipeId': 'itemId', 'AuthorId': 'userId'})

d = (dict(enumerate(df['userId'].unique())))
new_d = {}
new_d = {value:key for key, value in d.items()}
df['userId'] = df['userId'].map(new_d)

d = (dict(enumerate(df['itemId'].unique())))
new_d = {}
new_d = {value:key for key, value in d.items()}
df['itemId'] = df['itemId'].map(new_d)

train, test = train_test_split(df[(df['userId']<10000) & (df['itemId']<3000)], test_size=0.2)

coverage = 0
mae_value = IB_MAE(train, test, 800)   # Returning MAE for each fold.
print("MAE =", mae_value, ", Coverage = " + str(100-((coverage*100)/test.shape[0])) + "%")

100%|██████████████████████████████████████████████████████████████████████████████| 2817/2817 [16:46<00:00,  2.80it/s]
3021it [00:27, 109.52it/s]

MAE = 0.8073645224326266 , Coverage = 100.0%
